In [1]:
import pandas as pd

In [2]:
path= r"Documents/ESTÁGIO LGHM - BIOINFO - CAMILA/DATASETS/TCGA_mutations_stomach_primary_tumor.json"

In [6]:
json = pd.read_json(path, orient='records')

In [11]:
json

,consequence,mutation_subtype,ssm_id,genomic_dna_change
0,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,7fc63c99-6bf6-5184-abeb-4d31ec07c4e9,chr7:g.151360230C>T
1,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,4f4cf272-45d5-5855-a1b2-d0cc235a6844,chr14:g.63204918G>A
2,[{'transcript': {'annotation': {'polyphen_impa...,Small insertion,64aceac6-ceb3-5018-b99a-389e4072b074,chr3:g.51419993_51419994insT
3,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,cb959ed8-7715-5932-b6b5-8616c2a52e09,chr8:g.138242991A>C
4,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,17b015a1-6b9f-5b4c-912a-a367ef17c436,chr10:g.77805688C>T
...,...,...,...,...
154868,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,70ba1940-6f1b-5b1e-b232-30916360dc37,chr20:g.38771746G>A
154869,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,c2af4aee-143b-5d56-97d8-320c124b209d,chr14:g.80899950G>T
154870,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,479423c5-356b-522f-a054-aba333618095,chr10:g.84141510G>A
154871,[{'transcript': {'annotation': {'polyphen_impa...,Single base substitution,5a49aa91-f840-5c42-a160-1a867d1a1922,chr2:g.102375972A>C


In [50]:
lista = []
for i in range(154873):
    lista.append(json['consequence'][i][0]['transcript']['gene']['symbol'])

lista.count('CFTR')

21

In [79]:
indices = [idx for idx, gene in enumerate(lista) if gene =='CFTR']

In [80]:
indices

[16653,
 27178,
 35740,
 36948,
 37062,
 39480,
 55896,
 80049,
 81520,
 81833,
 103592,
 108816,
 115173,
 118551,
 121087,
 129633,
 130505,
 140813,
 150716,
 152498,
 154664]